<a href="https://colab.research.google.com/github/Pruthviraj141/project141/blob/main/Copy_of_pccoeipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from imblearn.over_sampling import SMOTE
import lightgbm as lgb
import joblib

In [ ]:
# Load data
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')  # Assuming similar test data

<ipython-input-3-4f31dd9fbe56>:2: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv('train.csv')


In [ ]:
def preprocess_data(df):
    df = df.drop(['PlayerID', 'GameName'], axis=1)  # Remove non-predictive IDs

    # Convert version strings to numerical representations
    version_cols = ['GameVersion', 'AntiCheatVersion', 'SecuritySignatureVersion',
                   'EmbeddedBrowserVersion', 'OSVersionDetails']

    for col in version_cols:
        # Handle missing values by filling with '0.0.0' (or any placeholder)
        df[col] = df[col].fillna('0.0.0')

        # Convert version strings to numerical values
        df[col] = df[col].apply(lambda x: sum([int(n) if n.isdigit() else 0 for n in str(x).split('.')]))

    # Label encode categorical features
    cat_cols = df.select_dtypes(include=['object']).columns
    for col in cat_cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))

    # Feature engineering
    df['SecurityTools_Ratio'] = df['SecurityToolsEnabled'] / (df['SecurityToolsInstalled'] + 1e-5)
    df['OS_Complexity'] = df['OSBuildNum'] * df['OSBuildRev']

    return df

In [ ]:
train_df = preprocess_data(train_df)
test_df = preprocess_data(test_df)

# Handle missing values in numerical columns
train_df = train_df.fillna(train_df.median(numeric_only=True))
test_df = test_df.fillna(test_df.median(numeric_only=True))


In [ ]:
# Split data
X = train_df.drop('CheatingFlag', axis=1)
y = train_df['CheatingFlag']
X_test = test_df

In [ ]:
# Handle class imbalance with SMOTE
smote = SMOTE(sampling_strategy='auto', random_state=42)  # Use 'auto' for default behavior
X_res, y_res = smote.fit_resample(X, y)

In [ ]:
params = {
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'scale_pos_weight': len(y[y==0])/len(y[y==1]),
    'seed': 42,
    'verbose': -1  # Verbosity level (-1 for no output, 1 for progress)
}

In [ ]:
# Stratified K-Fold Cross Validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof_preds = np.zeros(X_res.shape[0])
test_preds = []

In [ ]:
# LightGBM parameters (optimized for imbalance)
params = {
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'scale_pos_weight': len(y[y==0])/len(y[y==1]),
    'seed': 42,
    'verbose': 1  # Verbosity level (1 for progress updates)
}

# Stratified K-Fold Cross Validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof_preds = np.zeros(X_res.shape[0])
test_preds = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X_res, y_res)):
    X_train, X_val = X_res.iloc[train_idx], X_res.iloc[val_idx]
    y_train, y_val = y_res.iloc[train_idx], y_res.iloc[val_idx]

    train_data = lgb.Dataset(X_train, label=y_train)
    val_data = lgb.Dataset(X_val, label=y_val)

    model = lgb.train(
        params,
        train_data,
        num_boost_round=10000,
        valid_sets=[val_data],
        callbacks=[lgb.early_stopping(stopping_rounds=100)]  # Early stopping here
    )

    oof_preds[val_idx] = model.predict(X_val, num_iteration=model.best_iteration)
    test_preds.append(model.predict(X_test, num_iteration=model.best_iteration))

    joblib.dump(model, f'lgbm_fold{fold+1}.pkl')

# Ensemble test predictions
final_preds = np.mean(test_preds, axis=0)
test_preds_binary = (final_preds > 0.5).astype(int)

# Validation metrics
print(f"\nFinal Validation AUC: {roc_auc_score(y_res, oof_preds):.4f}")
print(f"Validation Accuracy: {accuracy_score(y_res, (oof_preds > 0.5).astype(int)):.4f}")
print(f"Validation F1: {f1_score(y_res, (oof_preds > 0.5).astype(int)):.4f}")

# Feature importance
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': model.feature_importance()
}).sort_values('Importance', ascending=False)

print("\nTop 20 Features:")
print(feature_importance.head(20))

# # Generate submission
# submission = pd.DataFrame({
#     "PlayerID": test["PlayerID"],
#     "CheatingFlag": test_preds_binary
# })
# submission.to_csv("submission_optimized.csv", index=False)

[LightGBM] [Info] Number of positive: 178504, number of negative: 178504
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.128233 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5236
[LightGBM] [Info] Number of data points in the train set: 357008, number of used features: 77
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[2091]	valid_0's auc: 0.719674
[LightGBM] [Info] Number of positive: 178504, number of negative: 178504
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.124823 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5242
[LightGBM] [Info] Number of data 